# The following notebook uses the dataframe on which tracking has been performed and uses the spots coordinates to extract activity from alternate channels. 

In [1]:
from os import path
import pandas as pd
from IPython.display import display
import numpy as np 
import sys 
import time 
import zarr
import os

sys.path.append('../src/')

from extract_pixel_data import Extractor

### Do not change the code in the cell below

In [2]:
# This assumes that your notebook is inside 'Jupyter Notebooks', which is at the same level as 'test_data'
base_dir = os.path.join(os.path.dirname(os.path.abspath("__file__")), '..', 'test_data')

zarr_directory = 'zarr_file/all_channels_data'
zarr_full_path = os.path.join(base_dir, zarr_directory)

input_directory = 'datasets'
input_file_name = 'track_df_c3_cleaned.pkl'
input_directory_full = os.path.join(base_dir,input_directory, input_file_name)

output_directory = 'datasets'
output_file_name = 'track_df_cleaned_final_full.pkl'
output_directory_full = os.path.join(base_dir,output_directory, output_file_name)

In [3]:
track_df = pd.read_pickle(input_directory_full)
# read the zarr file which contains the data of all three channels 
z = zarr.open(zarr_full_path, mode='r')

In [4]:
track_df

,frame,index,amplitude,mu_x,mu_y,mu_z,sigma_x,sigma_y,sigma_z,frame_y,tree_id,track_id
0,0,0,206.593249,11.0,12.0,0.0,2.0,2.0,4.0,0,0,0
1,0,1,191.000000,36.0,88.0,1.0,2.0,1.0,2.0,0,1,1
2,0,2,231.007679,40.0,141.0,1.0,2.0,2.0,2.0,0,2,2
3,0,3,185.152538,40.0,58.0,1.0,2.0,1.0,3.0,0,3,3
4,0,4,202.000000,46.0,73.0,1.0,2.0,2.0,3.0,0,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...
28371,129,163,213.439065,135.0,131.0,73.0,2.0,3.0,4.0,129,16373,16373
28372,129,164,200.000000,136.0,147.0,72.0,2.0,1.0,5.0,129,16374,16374
28373,129,165,177.830039,160.0,133.0,72.0,2.0,2.0,3.0,129,16375,16375
28374,129,166,179.000000,201.0,207.0,71.0,2.0,1.0,3.0,129,16376,16376


In [5]:
z.info

Type,zarr.core.Array
Data type,uint16
Shape,"(130, 3, 75, 258, 275)"
Chunk shape,"(1, 1, 75, 258, 275)"
Order,C
Read-only,True
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,zarr.storage.DirectoryStore
No. bytes,4150575000 (3.9G)
No. bytes stored,1126870917 (1.0G)
Storage ratio,3.7


In [6]:
track_df['frame'].nunique()

130

## In the below cell the Extractor object is created 

In [7]:
# Create an extractor object with required parameters to properly run the methods in next steps 
# radii is the expected radius of the CME sites (will be used for calculations which need fixed radius) 
# radii_col_name is a list of the variable sigma estimated by gaussian fitting (note: the convention for the list is z,y,x)
# n_jobs is the cores to be used. (parallel processing for gaussian fitting on the other two channels could be done to speed up the process)
extractor = Extractor(z, dataframe = track_df, radii=[4,2,2], frame_col_name = 'frame', 
                      radi_col_name = ['sigma_z', 'sigma_y', 'sigma_x'], n_jobs =2)

# Extract Information for Channel 3 (Clathrin)

## Extracting voxel sum

In [8]:
# pass the channel for which voxel sum is needed and the coordinates around which voxel sum is supposed to be calculated
# convention for coords [z,y,x] 
# convention for channel is 1 for channel 1, 2 for channel 2 and so on 
# channel number is to passed according to whichever channel we want to extract the data for
start_time = time.time()
voxel_sum_array_3, _, adj_voxel_sum_3 = extractor.voxel_sum_fixed_background(center_col_names = ['mu_z', 'mu_y', 'mu_x'], channel = 3,
                                                                            background_radius=[1,1,1])
end_time = time.time()
print('time taken (seconds)', end_time - start_time)

current frame is 0
current frame is 1
current frame is 2
current frame is 3
current frame is 4
current frame is 5
current frame is 6
current frame is 7
current frame is 8
current frame is 9
current frame is 10
current frame is 11
current frame is 12
current frame is 13
current frame is 14
current frame is 15
current frame is 16
current frame is 17
current frame is 18
current frame is 19
current frame is 20
current frame is 21
current frame is 22
current frame is 23
current frame is 24
current frame is 25
current frame is 26
current frame is 27
current frame is 28
current frame is 29
current frame is 30
current frame is 31
current frame is 32
current frame is 33
current frame is 34
current frame is 35
current frame is 36
current frame is 37
current frame is 38
current frame is 39
current frame is 40
current frame is 41
current frame is 42
current frame is 43
current frame is 44
current frame is 45
current frame is 46
current frame is 47
current frame is 48
current frame is 49
current fr

In [9]:
# pass the channel for which pixel values are needed and the coordinates around which values are supposed to be calculated
# convention for coords [z,y,x] 
# convention for channel is 1 for channel 1, 2 for channel 2 and so on 
start_time = time.time()
offset = [0,0]
col_names = ['mu_z', 'mu_y', 'mu_x']
mean,maximum,minimum,pixel_values,max_loc = extractor.extract_pixels_data_variable_bd(center_col_names = col_names, channel = 3)

end_time = time.time()
print('time taken (seconds)', end_time - start_time)

current frame number is 0
current frame number is 1
current frame number is 2
current frame number is 3
current frame number is 4
current frame number is 5
current frame number is 6
current frame number is 7
current frame number is 8
current frame number is 9
current frame number is 10
current frame number is 11
current frame number is 12
current frame number is 13
current frame number is 14
current frame number is 15
current frame number is 16
current frame number is 17
current frame number is 18
current frame number is 19
current frame number is 20
current frame number is 21
current frame number is 22
current frame number is 23
current frame number is 24
current frame number is 25
current frame number is 26
current frame number is 27
current frame number is 28
current frame number is 29
current frame number is 30
current frame number is 31
current frame number is 32
current frame number is 33
current frame number is 34
current frame number is 35
current frame number is 36
current fra

In [10]:
max_loc = np.array(max_loc)

In [11]:
track_df['c3_mean_amp'] = mean
track_df['c3_voxel_sum'] = voxel_sum_array_3
track_df['c3_voxel_sum_adjusted'] = adj_voxel_sum_3 
track_df['c3_peak_amp'] = maximum 
track_df['c3_peak_x'] = max_loc[:,2]
track_df['c3_peak_y'] = max_loc[:,1]
track_df['c3_peak_z'] = max_loc[:,0]

In [12]:
track_df

,frame,index,amplitude,mu_x,mu_y,mu_z,sigma_x,sigma_y,sigma_z,frame_y,tree_id,track_id,c3_mean_amp,c3_voxel_sum,c3_voxel_sum_adjusted,c3_peak_amp,c3_peak_x,c3_peak_y,c3_peak_z
0,0,0,206.593249,11.0,12.0,0.0,2.0,2.0,4.0,0,0,0,163.682353,7997,645.727273,207,12,13,0
1,0,1,191.000000,36.0,88.0,1.0,2.0,1.0,2.0,0,1,1,138.233333,10778,471.529412,196,36,89,1
2,0,2,231.007679,40.0,141.0,1.0,2.0,2.0,2.0,0,2,2,153.940000,12742,1473.764706,235,40,141,1
3,0,3,185.152538,40.0,58.0,1.0,2.0,1.0,3.0,0,3,3,141.280000,10928,577.411765,189,41,59,1
4,0,4,202.000000,46.0,73.0,1.0,2.0,2.0,3.0,0,4,4,147.768000,11688,690.941176,207,47,74,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28371,129,163,213.439065,135.0,131.0,73.0,2.0,3.0,4.0,129,16373,16373,164.841837,15010,1188.971429,215,135,130,73
28372,129,164,200.000000,136.0,147.0,72.0,2.0,1.0,5.0,129,16374,16374,162.649573,17798,238.357143,205,137,148,73
28373,129,165,177.830039,160.0,133.0,72.0,2.0,2.0,3.0,129,16375,16375,143.660000,16187,1172.000000,180,160,133,71
28374,129,166,179.000000,201.0,207.0,71.0,2.0,1.0,3.0,129,16376,16376,130.676190,16674,-701.346939,184,202,208,73


# Extract information for Channel 2 

In [13]:
# pass the channel for which pixel values are needed and the coordinates around which values are supposed to be calculated
# convention for coords [z,y,x] 
# convention for channel is 1 for channel 1, 2 for channel 2 and so on 
start_time = time.time()
offset = [0,0]
col_names = ['mu_z', 'mu_y', 'mu_x']
mean,maximum,minimum,pixel_values,max_loc = extractor.extract_pixels_data_variable_bd(center_col_names = col_names, channel = 2)

end_time = time.time()
print('time taken (seconds)', end_time - start_time)

current frame number is 0
current frame number is 1
current frame number is 2
current frame number is 3
current frame number is 4
current frame number is 5
current frame number is 6
current frame number is 7
current frame number is 8
current frame number is 9
current frame number is 10
current frame number is 11
current frame number is 12
current frame number is 13
current frame number is 14
current frame number is 15
current frame number is 16
current frame number is 17
current frame number is 18
current frame number is 19
current frame number is 20
current frame number is 21
current frame number is 22
current frame number is 23
current frame number is 24
current frame number is 25
current frame number is 26
current frame number is 27
current frame number is 28
current frame number is 29
current frame number is 30
current frame number is 31
current frame number is 32
current frame number is 33
current frame number is 34
current frame number is 35
current frame number is 36
current fra

In [14]:
max_loc = np.array(max_loc)

## Finding peak pixel for Channel 2 as not sure about the offset between Channel 3 and Channel 2
**Reason for offset is being shot from different cameras**

In [15]:
track_df['c2_amp'] = mean
track_df['c2_peak'] = maximum
track_df['c2_peak_x'] = max_loc[:,2]
track_df['c2_peak_y'] = max_loc[:,1]
track_df['c2_peak_z'] = max_loc[:,0]

In [16]:
track_df

,frame,index,amplitude,mu_x,mu_y,mu_z,sigma_x,sigma_y,sigma_z,frame_y,...,c3_voxel_sum_adjusted,c3_peak_amp,c3_peak_x,c3_peak_y,c3_peak_z,c2_amp,c2_peak,c2_peak_x,c2_peak_y,c2_peak_z
0,0,0,206.593249,11.0,12.0,0.0,2.0,2.0,4.0,0,...,645.727273,207,12,13,0,122.947826,152,9,14,2
1,0,1,191.000000,36.0,88.0,1.0,2.0,1.0,2.0,0,...,471.529412,196,36,89,1,116.233333,143,34,87,2
2,0,2,231.007679,40.0,141.0,1.0,2.0,2.0,2.0,0,...,1473.764706,235,40,141,1,122.330000,142,41,142,0
3,0,3,185.152538,40.0,58.0,1.0,2.0,1.0,3.0,0,...,577.411765,189,41,59,1,117.626667,142,42,59,0
4,0,4,202.000000,46.0,73.0,1.0,2.0,2.0,3.0,0,...,690.941176,207,47,74,1,115.112000,141,45,74,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28371,129,163,213.439065,135.0,131.0,73.0,2.0,3.0,4.0,129,...,1188.971429,215,135,130,73,126.480952,158,135,131,72
28372,129,164,200.000000,136.0,147.0,72.0,2.0,1.0,5.0,129,...,238.357143,205,137,148,73,118.616667,144,138,148,70
28373,129,165,177.830039,160.0,133.0,72.0,2.0,2.0,3.0,129,...,1172.000000,180,160,133,71,120.153333,146,161,135,70
28374,129,166,179.000000,201.0,207.0,71.0,2.0,1.0,3.0,129,...,-701.346939,184,202,208,73,115.380952,140,203,207,68


## Finding mean value around the peak pixel value for channel 2 

In [17]:
start_time = time.time()
col_names = ['c2_peak_z', 'c2_peak_y', 'c2_peak_x']
peak_mean,maxima,_,_,_ = extractor.extract_pixels_data_variable_bd(center_col_names = col_names, 
                                                     channel = 2)
end_time = time.time()
print('time taken (seconds)', end_time - start_time)

current frame number is 0
current frame number is 1
current frame number is 2
current frame number is 3
current frame number is 4
current frame number is 5
current frame number is 6
current frame number is 7
current frame number is 8
current frame number is 9
current frame number is 10
current frame number is 11
current frame number is 12
current frame number is 13
current frame number is 14
current frame number is 15
current frame number is 16
current frame number is 17
current frame number is 18
current frame number is 19
current frame number is 20
current frame number is 21
current frame number is 22
current frame number is 23
current frame number is 24
current frame number is 25
current frame number is 26
current frame number is 27
current frame number is 28
current frame number is 29
current frame number is 30
current frame number is 31
current frame number is 32
current frame number is 33
current frame number is 34
current frame number is 35
current frame number is 36
current fra

In [18]:
track_df['c2_peak_mean'] = peak_mean

In [19]:
track_df

,frame,index,amplitude,mu_x,mu_y,mu_z,sigma_x,sigma_y,sigma_z,frame_y,...,c3_peak_amp,c3_peak_x,c3_peak_y,c3_peak_z,c2_amp,c2_peak,c2_peak_x,c2_peak_y,c2_peak_z,c2_peak_mean
0,0,0,206.593249,11.0,12.0,0.0,2.0,2.0,4.0,0,...,207,12,13,0,122.947826,152,9,14,2,125.500000
1,0,1,191.000000,36.0,88.0,1.0,2.0,1.0,2.0,0,...,196,36,89,1,116.233333,143,34,87,2,114.826667
2,0,2,231.007679,40.0,141.0,1.0,2.0,2.0,2.0,0,...,235,40,141,1,122.330000,142,41,142,0,122.853333
3,0,3,185.152538,40.0,58.0,1.0,2.0,1.0,3.0,0,...,189,41,59,1,117.626667,142,42,59,0,118.016667
4,0,4,202.000000,46.0,73.0,1.0,2.0,2.0,3.0,0,...,207,47,74,1,115.112000,141,45,74,3,115.502857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28371,129,163,213.439065,135.0,131.0,73.0,2.0,3.0,4.0,129,...,215,135,130,73,126.480952,158,135,131,72,126.195918
28372,129,164,200.000000,136.0,147.0,72.0,2.0,1.0,5.0,129,...,205,137,148,73,118.616667,144,138,148,70,122.700000
28373,129,165,177.830039,160.0,133.0,72.0,2.0,2.0,3.0,129,...,180,160,133,71,120.153333,146,161,135,70,118.211429
28374,129,166,179.000000,201.0,207.0,71.0,2.0,1.0,3.0,129,...,184,202,208,73,115.380952,140,203,207,68,118.647619


## Finding voxel sum around peak for channel 2 

In [21]:
# pass the channel for which voxel sum is needed and the coordinates around which voxel sum is supposed to be calculated
# convention for coords [z,y,x] 
# convention for channel is 1 for channel 1, 2 for channel 2 and so on 
start_time = time.time()
voxel_sum_array_2, _, adj_voxel_sum_array_2 = extractor.voxel_sum_fixed_background(center_col_names = ['c2_peak_z', 'c2_peak_y', 'c2_peak_x'],
                                       channel = 2, background_radius = [1,1,1])
end_time = time.time()
print('time taken (seconds)', end_time - start_time)

current frame is 0
current frame is 1
current frame is 2
current frame is 3
current frame is 4
current frame is 5
current frame is 6
current frame is 7
current frame is 8
current frame is 9
current frame is 10
current frame is 11
current frame is 12
current frame is 13
current frame is 14
current frame is 15
current frame is 16
current frame is 17
current frame is 18
current frame is 19
current frame is 20
current frame is 21
current frame is 22
current frame is 23
current frame is 24
current frame is 25
current frame is 26
current frame is 27
current frame is 28
current frame is 29
current frame is 30
current frame is 31
current frame is 32
current frame is 33
current frame is 34
current frame is 35
current frame is 36
current frame is 37
current frame is 38
current frame is 39
current frame is 40
current frame is 41
current frame is 42
current frame is 43
current frame is 44
current frame is 45
current frame is 46
current frame is 47
current frame is 48
current frame is 49
current fr

In [22]:
#calculated around the peak value coordinates
track_df['c2_voxel_sum'] = voxel_sum_array_2
track_df['c2_voxel_sum_adjusted'] = adj_voxel_sum_array_2

In [23]:
track_df

,frame,index,amplitude,mu_x,mu_y,mu_z,sigma_x,sigma_y,sigma_z,frame_y,...,c3_peak_y,c3_peak_z,c2_amp,c2_peak,c2_peak_x,c2_peak_y,c2_peak_z,c2_peak_mean,c2_voxel_sum,c2_voxel_sum_adjusted
0,0,0,206.593249,11.0,12.0,0.0,2.0,2.0,4.0,0,...,13,0,122.947826,152,9,14,2,125.500000,6646,418.235294
1,0,1,191.000000,36.0,88.0,1.0,2.0,1.0,2.0,0,...,89,1,116.233333,143,34,87,2,114.826667,10934,-79.538462
2,0,2,231.007679,40.0,141.0,1.0,2.0,2.0,2.0,0,...,141,1,122.330000,142,41,142,0,122.853333,7836,-13.931034
3,0,3,185.152538,40.0,58.0,1.0,2.0,1.0,3.0,0,...,59,1,117.626667,142,42,59,0,118.016667,7556,194.896552
4,0,4,202.000000,46.0,73.0,1.0,2.0,2.0,3.0,0,...,74,1,115.112000,141,45,74,3,115.502857,12972,113.636364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28371,129,163,213.439065,135.0,131.0,73.0,2.0,3.0,4.0,129,...,130,73,126.480952,158,135,131,72,126.195918,14511,593.727273
28372,129,164,200.000000,136.0,147.0,72.0,2.0,1.0,5.0,129,...,148,73,118.616667,144,138,148,70,122.700000,15498,-359.103448
28373,129,165,177.830039,160.0,133.0,72.0,2.0,2.0,3.0,129,...,133,71,120.153333,146,161,135,70,118.211429,15080,195.586207
28374,129,166,179.000000,201.0,207.0,71.0,2.0,1.0,3.0,129,...,208,73,115.380952,140,203,207,68,118.647619,15224,-210.482759


### Gaussian Fitting for Channel 2 around peak values 

In [25]:
#Calculating the gaussian fitting estimates around peak coords for channel 2 
#expected sigma value needed for gaussian fitting 
#set all frames to False for processing limited frames 
#max_frames determines the frames to be processed if all_frames is false
start_time = time.time()
channel2_gaussians_df = extractor.run_parallel_frame_processing(expected_sigma = [4,2,2], 
                                        center_col_name = ['c2_peak_z', 'c2_peak_y', 'c2_peak_x'], 
                                       dist_between_spots = 10 , channel = 2,  max_frames =  2, all_frames = True)
end_time = time.time()
print('time taken (seconds)', end_time - start_time)

Processing frames: 100%|██████████████████████| 130/130 [03:47<00:00,  1.75s/it]


10%(23 of 221)
20%(45 of 221)
30%(67 of 221)
40%(89 of 221)
50%(111 of 221)
60%(133 of 221)
70%(155 of 221)
80%(177 of 221)
90%(199 of 221)
100%(221 of 221)
(221, 7)
the number of times the gaussian fitting worked was 221 and the number of times the gaussian did not fit was 0
10%(24 of 237)
20%(48 of 237)
30%(72 of 237)
40%(95 of 237)
50%(119 of 237)
60%(143 of 237)
70%(166 of 237)
80%(190 of 237)
90%(214 of 237)
100%(237 of 237)
(237, 7)
the number of times the gaussian fitting worked was 237 and the number of times the gaussian did not fit was 0
10%(25 of 250)
20%(50 of 250)
30%(75 of 250)
40%(100 of 250)
50%(125 of 250)
60%(150 of 250)
70%(175 of 250)
80%(200 of 250)
90%(225 of 250)
100%(250 of 250)
(250, 7)
the number of times the gaussian fitting worked was 250 and the number of times the gaussian did not fit was 0
10%(23 of 229)
20%(46 of 229)
30%(69 of 229)
40%(92 of 229)
50%(115 of 229)
60%(138 of 229)
70%(161 of 229)
80%(184 of 229)
90%(207 of 229)
100%(229 of 229)
(229, 7)
th

In [26]:
track_df['c2_gaussian_amp'] = channel2_gaussians_df['amplitude']
track_df['c2_mu_x'] = channel2_gaussians_df['mu_x']
track_df['c2_mu_y'] = channel2_gaussians_df['mu_y']
track_df['c2_mu_z'] = channel2_gaussians_df['mu_z']
track_df['c2_sigma_x'] = channel2_gaussians_df['sigma_x']
track_df['c2_sigma_y'] = channel2_gaussians_df['sigma_y']
track_df['c2_sigma_z'] = channel2_gaussians_df['sigma_z']

In [27]:
track_df

,frame,index,amplitude,mu_x,mu_y,mu_z,sigma_x,sigma_y,sigma_z,frame_y,...,c2_peak_mean,c2_voxel_sum,c2_voxel_sum_adjusted,c2_gaussian_amp,c2_mu_x,c2_mu_y,c2_mu_z,c2_sigma_x,c2_sigma_y,c2_sigma_z
0,0,0,206.593249,11.0,12.0,0.0,2.0,2.0,4.0,0,...,125.500000,6646,418.235294,153.666667,9.0,13.0,1.0,1.0,4.0,1.0
1,0,1,191.000000,36.0,88.0,1.0,2.0,1.0,2.0,0,...,114.826667,10934,-79.538462,138.000000,33.0,87.0,2.0,2.0,3.0,3.0
2,0,2,231.007679,40.0,141.0,1.0,2.0,2.0,2.0,0,...,122.853333,7836,-13.931034,138.662341,40.0,141.0,-4.0,4.0,2.0,22.0
3,0,3,185.152538,40.0,58.0,1.0,2.0,1.0,3.0,0,...,118.016667,7556,194.896552,141.721258,41.0,58.0,-2.0,2.0,2.0,7.0
4,0,4,202.000000,46.0,73.0,1.0,2.0,2.0,3.0,0,...,115.502857,12972,113.636364,137.294351,45.0,73.0,3.0,3.0,3.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28371,129,163,213.439065,135.0,131.0,73.0,2.0,3.0,4.0,129,...,126.195918,14511,593.727273,155.607698,134.0,130.0,71.0,2.0,2.0,4.0
28372,129,164,200.000000,136.0,147.0,72.0,2.0,1.0,5.0,129,...,122.700000,15498,-359.103448,140.027180,138.0,149.0,69.0,3.0,5.0,4.0
28373,129,165,177.830039,160.0,133.0,72.0,2.0,2.0,3.0,129,...,118.211429,15080,195.586207,141.754593,160.0,134.0,69.0,4.0,2.0,5.0
28374,129,166,179.000000,201.0,207.0,71.0,2.0,1.0,3.0,129,...,118.647619,15224,-210.482759,138.333333,206.0,211.0,67.0,7.0,17.0,6.0


# Extract information for channel 1

In [28]:
offset = [0,0]
col_names = ['mu_z', 'mu_y', 'mu_x']
radi_list = ['sigma_z', 'sigma_y', 'sigma_x']
mean,maximum,minimum,pixel_values,max_loc = extractor.extract_pixels_data_variable_bd(center_col_names = col_names, channel = 1)

current frame number is 0
current frame number is 1
current frame number is 2
current frame number is 3
current frame number is 4
current frame number is 5
current frame number is 6
current frame number is 7
current frame number is 8
current frame number is 9
current frame number is 10
current frame number is 11
current frame number is 12
current frame number is 13
current frame number is 14
current frame number is 15
current frame number is 16
current frame number is 17
current frame number is 18
current frame number is 19
current frame number is 20
current frame number is 21
current frame number is 22
current frame number is 23
current frame number is 24
current frame number is 25
current frame number is 26
current frame number is 27
current frame number is 28
current frame number is 29
current frame number is 30
current frame number is 31
current frame number is 32
current frame number is 33
current frame number is 34
current frame number is 35
current frame number is 36
current fra

In [29]:
max_loc = np.array(max_loc)

In [30]:
track_df['c1_amp'] = mean
track_df['c1_peak'] = maximum
track_df['c1_peak_x'] = max_loc[:,2]
track_df['c1_peak_y'] = max_loc[:,1]
track_df['c1_peak_z'] = max_loc[:,0]

In [31]:
voxel_sum_array_1, _, adj_voxel_sum_array_1 = extractor.voxel_sum_fixed_background(center_col_names = ['mu_z', 'mu_y', 'mu_x'], channel = 1,
                                                                                  background_radius = [1,1,1])


current frame is 0
current frame is 1
current frame is 2
current frame is 3
current frame is 4
current frame is 5
current frame is 6
current frame is 7
current frame is 8
current frame is 9
current frame is 10
current frame is 11
current frame is 12
current frame is 13
current frame is 14
current frame is 15
current frame is 16
current frame is 17
current frame is 18
current frame is 19
current frame is 20
current frame is 21
current frame is 22
current frame is 23
current frame is 24
current frame is 25
current frame is 26
current frame is 27
current frame is 28
current frame is 29
current frame is 30
current frame is 31
current frame is 32
current frame is 33
current frame is 34
current frame is 35
current frame is 36
current frame is 37
current frame is 38
current frame is 39
current frame is 40
current frame is 41
current frame is 42
current frame is 43
current frame is 44
current frame is 45
current frame is 46
current frame is 47
current frame is 48
current frame is 49
current fr

In [32]:
#calculated around the peak value coordinates
track_df['c1_voxel_sum'] = voxel_sum_array_1
track_df['c1_voxel_sum_adjusted'] = adj_voxel_sum_array_1

In [33]:
track_df

,frame,index,amplitude,mu_x,mu_y,mu_z,sigma_x,sigma_y,sigma_z,frame_y,...,c2_sigma_x,c2_sigma_y,c2_sigma_z,c1_amp,c1_peak,c1_peak_x,c1_peak_y,c1_peak_z,c1_voxel_sum,c1_voxel_sum_adjusted
0,0,0,206.593249,11.0,12.0,0.0,2.0,2.0,4.0,0,...,1.0,4.0,1.0,175.329412,231,10,14,2,8395,135.363636
1,0,1,191.000000,36.0,88.0,1.0,2.0,1.0,2.0,0,...,2.0,3.0,3.0,157.133333,189,38,87,3,12758,-47.294118
2,0,2,231.007679,40.0,141.0,1.0,2.0,2.0,2.0,0,...,4.0,2.0,22.0,154.560000,198,40,140,3,12604,25.176471
3,0,3,185.152538,40.0,58.0,1.0,2.0,1.0,3.0,0,...,2.0,2.0,7.0,152.960000,196,42,59,3,12234,-298.941176
4,0,4,202.000000,46.0,73.0,1.0,2.0,2.0,3.0,0,...,3.0,3.0,6.0,160.240000,219,48,72,4,12659,-1.588235
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28371,129,163,213.439065,135.0,131.0,73.0,2.0,3.0,4.0,129,...,2.0,2.0,4.0,279.255102,389,133,133,73,25024,2235.142857
28372,129,164,200.000000,136.0,147.0,72.0,2.0,1.0,5.0,129,...,3.0,5.0,4.0,335.239316,432,135,148,70,36425,837.071429
28373,129,165,177.830039,160.0,133.0,72.0,2.0,2.0,3.0,129,...,4.0,2.0,5.0,217.033333,296,159,133,71,24469,1379.818182
28374,129,166,179.000000,201.0,207.0,71.0,2.0,1.0,3.0,129,...,7.0,17.0,6.0,187.447619,221,200,208,68,23992,-227.428571


In [34]:
track_df.shape

(28376, 41)

### Perform Guassian Fitting for channel 1

In [35]:
start_time = time.time()
channel1_gaussians_df = extractor.run_parallel_frame_processing(expected_sigma = [4,2,2], 
                                        center_col_name = ['c1_peak_z', 'c1_peak_y', 'c1_peak_x'], 
                                       dist_between_spots = 10, channel = 1,  max_frames =  2, all_frames = True)
end_time = time.time()
print('time taken (seconds)', end_time - start_time)

Processing frames: 100%|██████████████████████| 130/130 [04:14<00:00,  1.96s/it]


10%(23 of 221)
20%(45 of 221)
30%(67 of 221)
40%(89 of 221)
50%(111 of 221)
60%(133 of 221)
70%(155 of 221)
80%(177 of 221)
90%(199 of 221)
100%(221 of 221)
(221, 7)
the number of times the gaussian fitting worked was 221 and the number of times the gaussian did not fit was 0
10%(24 of 237)
20%(48 of 237)
30%(72 of 237)
40%(95 of 237)
50%(119 of 237)
60%(143 of 237)
70%(166 of 237)
80%(190 of 237)
90%(214 of 237)
100%(237 of 237)
(237, 7)
the number of times the gaussian fitting worked was 237 and the number of times the gaussian did not fit was 0
10%(25 of 246)
20%(50 of 246)
30%(74 of 246)
40%(99 of 246)
50%(123 of 246)
60%(148 of 246)
70%(173 of 246)
80%(197 of 246)
90%(222 of 246)
100%(246 of 246)
(246, 7)
the number of times the gaussian fitting worked was 246 and the number of times the gaussian did not fit was 0
10%(23 of 229)
20%(46 of 229)
30%(69 of 229)
40%(92 of 229)
50%(115 of 229)
60%(138 of 229)
70%(161 of 229)
80%(184 of 229)
90%(207 of 229)
100%(229 of 229)
(229, 7)
the

In [36]:
channel1_gaussians_df

,amplitude,mu_x,mu_y,mu_z,sigma_x,sigma_y,sigma_z,frame
0,229.333333,10.0,14.0,1.0,0.0,2.0,1.0,0
1,185.082663,37.0,86.0,7.0,3.0,3.0,10.0,0
2,196.333333,39.0,139.0,3.0,2.0,2.0,4.0,0
3,191.000000,43.0,58.0,3.0,4.0,2.0,3.0,0
4,214.000000,48.0,71.0,4.0,3.0,2.0,4.0,0
...,...,...,...,...,...,...,...,...
28371,387.333333,133.0,132.0,71.0,0.0,2.0,1.0,129
28372,427.700735,134.0,149.0,70.0,3.0,4.0,4.0,129
28373,293.865853,158.0,132.0,71.0,2.0,2.0,3.0,129
28374,216.000000,199.0,207.0,67.0,2.0,2.0,5.0,129


In [37]:
track_df['c1_gaussian_amp'] = channel1_gaussians_df['amplitude']
track_df['c1_mu_x'] = channel1_gaussians_df['mu_x']
track_df['c1_mu_y'] = channel1_gaussians_df['mu_y']
track_df['c1_mu_z'] = channel1_gaussians_df['mu_z']
track_df['c1_sigma_x'] = channel1_gaussians_df['sigma_x']
track_df['c1_sigma_y'] = channel1_gaussians_df['sigma_y']
track_df['c1_sigma_z'] = channel1_gaussians_df['sigma_z']

In [38]:
track_df

,frame,index,amplitude,mu_x,mu_y,mu_z,sigma_x,sigma_y,sigma_z,frame_y,...,c1_peak_z,c1_voxel_sum,c1_voxel_sum_adjusted,c1_gaussian_amp,c1_mu_x,c1_mu_y,c1_mu_z,c1_sigma_x,c1_sigma_y,c1_sigma_z
0,0,0,206.593249,11.0,12.0,0.0,2.0,2.0,4.0,0,...,2,8395,135.363636,229.333333,10.0,14.0,1.0,0.0,2.0,1.0
1,0,1,191.000000,36.0,88.0,1.0,2.0,1.0,2.0,0,...,3,12758,-47.294118,185.082663,37.0,86.0,7.0,3.0,3.0,10.0
2,0,2,231.007679,40.0,141.0,1.0,2.0,2.0,2.0,0,...,3,12604,25.176471,196.333333,39.0,139.0,3.0,2.0,2.0,4.0
3,0,3,185.152538,40.0,58.0,1.0,2.0,1.0,3.0,0,...,3,12234,-298.941176,191.000000,43.0,58.0,3.0,4.0,2.0,3.0
4,0,4,202.000000,46.0,73.0,1.0,2.0,2.0,3.0,0,...,4,12659,-1.588235,214.000000,48.0,71.0,4.0,3.0,2.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28371,129,163,213.439065,135.0,131.0,73.0,2.0,3.0,4.0,129,...,73,25024,2235.142857,387.333333,133.0,132.0,71.0,0.0,2.0,1.0
28372,129,164,200.000000,136.0,147.0,72.0,2.0,1.0,5.0,129,...,70,36425,837.071429,427.700735,134.0,149.0,70.0,3.0,4.0,4.0
28373,129,165,177.830039,160.0,133.0,72.0,2.0,2.0,3.0,129,...,71,24469,1379.818182,293.865853,158.0,132.0,71.0,2.0,2.0,3.0
28374,129,166,179.000000,201.0,207.0,71.0,2.0,1.0,3.0,129,...,68,23992,-227.428571,216.000000,199.0,207.0,67.0,2.0,2.0,5.0


83 of 203)
100%(203 of 203)
(203, 7)
the number of times the gaussian fitting worked was 203 and the number of times the gaussian did not fit was 0
10%(22 of 216)
20%(44 of 216)
30%(65 of 216)
40%(87 of 216)
50%(108 of 216)
60%(130 of 216)
70%(152 of 216)
80%(173 of 216)
90%(195 of 216)
100%(216 of 216)
(216, 7)
the number of times the gaussian fitting worked was 216 and the number of times the gaussian did not fit was 0
10%(23 of 229)
20%(46 of 229)
30%(69 of 229)
40%(92 of 229)
50%(115 of 229)
60%(138 of 229)
70%(161 of 229)
80%(184 of 229)
90%(207 of 229)
100%(229 of 229)
(229, 7)
the number of times the gaussian fitting worked was 229 and the number of times the gaussian did not fit was 0
10%(21 of 205)
20%(41 of 205)
30%(62 of 205)
40%(82 of 205)
50%(103 of 205)
60%(123 of 205)
70%(144 of 205)
80%(164 of 205)
90%(185 of 205)
100%(205 of 205)
(205, 7)
the number of times the gaussian fitting worked was 205 and the number of times the gaussian did not fit was 0
10%(23 of 228)
20%(46

In [39]:
track_df.to_pickle(output_directory_full)

# To test the following functions change the markdown files to code files 
1. Voxel sum using the variable sigma/radi values directly inferred from the dataset 
2. Extracting pixel values mean,max etc using fixed radi
3. Performing Gaussian Fitting on peak coords for single frame

### Function 1 (Voxel Sum)
start_time = time.time()
voxel_sum_array_variable, _ = extractor.voxel_sum_variable_bd(center_col_names = ['mu_z', 'mu_y', 'mu_x'], channel = 3)
end_time = time.time()
print('time taken (seconds)', end_time - start_time)

### Function 2 (Extracting Pixel Values)
start_time = time.time()
offset = [0,0]
col_names = ['mu_z', 'mu_y', 'mu_x']
radii = [4, 2, 2]
mean_v,maximum_v,minimum_v,pixel_values_v,max_loc_v = extractor.extract_pixels_data_fixed_bd(center_col_names = col_names, channel = 3)

end_time = time.time()
print('time taken (seconds)', end_time - start_time)

### Function 3 (Gaussian fitting on one frame for one channel)
df = extractor.gaussian_fitting_single_frame(expected_sigma = [4,2,2], 
                              center_col_names = ['c2_peak_z', 'c2_peak_y', 'c2_peak_x'],
                                      frame = 0, channel = 2, dist_between_spots = 10)